In [1]:
import sqlite3
import random
import time
import os
import re
import unidecode

In [2]:
conn = sqlite3.connect(os.path.join(os.getcwd(), "clientes.sqlite3"))  # Base de datos en memoria
cur = conn.cursor()

In [3]:
cur.execute('''CREATE TABLE Clientes (
                id INTEGER PRIMARY KEY,
                nombre TEXT,
                ciudad TEXT)''')

In [4]:
def borrar_diacritico(texto):
    
    keep = 'çÇ'
    
    return re.sub(f"([^{re.escape(keep)}]+)", lambda m: unidecode.unidecode(m.group(0)), texto)

In [5]:
def generar_nombre():
    
    nombres = [
        "Alejandro", "Beatriz", "Carlos", "Diana", "Esteban", "Fernanda", "Gabriel", "Hilda", 
        "Ignacio", "Jimena", "Karla", "Luis", "Marta", "Nicolás", "Olga", "Pablo", "Queralt", 
        "Rafael", "Sara", "Tomás", "Úrsula", "Valeria", "Walter", "Ximena", "Yolanda", "Zacarías",
        "Andrés", "Bárbara", "César", "Daniela", "Emilio", "Felipe", "Gloria", "Héctor", "Iván", 
        "Juliana", "Kevin", "Lorena", "Miguel", "Natalia", "Oscar", "Patricia", "Raúl", 
        "Silvia", "Teodoro", "Uriel", "Verónica", "Wilfredo", "Xavier", "Yago", "Zoé"
    ]
    
    apellidos = [
        "García", "Fernández", "López", "Martínez", "Rodríguez", "Pérez", "Sánchez", "Gómez", 
        "Martín", "Jiménez", "Ruiz", "Hernández", "Díaz", "Moreno", "Muñoz", "Álvarez", "Romero", 
        "Gutiérrez", "Alonso", "Navarro", "Torres", "Domínguez", "Vázquez", "Ramos", "Gil", 
        "Serrano", "Molina", "Blanco", "Castro", "Ortiz", "Rubio", "Delgado", "Méndez", 
        "Iglesias", "Cruz", "Santos", "Ortega", "Mora", "Vera", "Peña", "Carrasco", "Flores", 
        "Cabrera", "Reyes", "Nieto", "Herrera", "Lara", "Marín", "Soto", "Campos", "León"
    ]

    return borrar_diacritico(random.choice(nombres)) + ' ' + borrar_diacritico(random.choice(apellidos))

In [6]:
ciudades = [
    "Buenos Aires", "Catamarca", "Chaco", "Chubut", "Córdoba", "Corrientes", "Entre Ríos", 
    "Formosa", "Jujuy", "La Pampa", "La Rioja", "Mendoza", "Misiones", "Neuquén", 
    "Río Negro", "Salta", "San Juan", "San Luis", "Santa Cruz", "Santa Fe", "Santiago del Estero", 
    "Tierra del Fuego", "Tucumán", "Ciudad Autónoma de Buenos Aires"
]


for i in range(1000000):
    nombre = generar_nombre()
    ciudad = borrar_diacritico(random.choice(ciudades))
    cur.execute('INSERT INTO Clientes (nombre, ciudad) VALUES (?, ?)', (nombre, ciudad))

In [7]:
conn.commit()

In [8]:
ciudad_busqueda = 'Jujuy'
consulta = f"SELECT * FROM Clientes WHERE ciudad = '{ciudad_busqueda}'"

start_time = time.time()
cur.execute(consulta)
resultados_sin_indice = cur.fetchall()
tiempo_sin_indice = time.time() - start_time

print(f'Tiempo de ejecución sin índice: {tiempo_sin_indice:.6f} segundos')
print(f'Número de resultados sin índice: {len(resultados_sin_indice)}')


Tiempo de ejecución sin índice: 0.383903 segundos
Número de resultados sin índice: 42243


In [9]:
print("\nPlan de ejecución sin índice:")
cur.execute(f'EXPLAIN QUERY PLAN {consulta}')
for row in cur.fetchall():
    print(row)


Plan de ejecución sin índice:
(2, 0, 0, 'SCAN Clientes')


In [10]:
# Paso 4: Crear un índice en la columna ciudad
cur.execute('CREATE INDEX idx_ciudad ON Clientes (ciudad)')
conn.commit()

In [11]:
start_time = time.time()
cur.execute(consulta)
resultados_con_indice = cur.fetchall()
tiempo_con_indice = time.time() - start_time


print(f'Tiempo de ejecución con índice: {tiempo_con_indice:.6f} segundos')
print(f'Número de resultados con índice: {len(resultados_con_indice)}')

Tiempo de ejecución con índice: 0.176803 segundos
Número de resultados con índice: 42243


In [12]:
print("\nPlan de ejecución con índice:")
cur.execute(f'EXPLAIN QUERY PLAN {consulta}')
for row in cur.fetchall():
    print(row)

# Cerrar la conexión
conn.close()


Plan de ejecución con índice:
(3, 0, 0, 'SEARCH Clientes USING INDEX idx_ciudad (ciudad=?)')
